# Two-Pane Refactor Automation (Binder retained)

This notebook automates applying refactor steps while retaining the user-facing term "Binder". It syncs with GitHub, discovers files, runs guarded transforms, formats/lints, tests, bumps version, and pushes changes.


In [14]:
# 1. Set Up Environment and Load Repo Settings
import os, sys, json, subprocess, platform
from pathlib import Path

# Configurable env
REPO_PATH = Path(os.getenv("REPO_PATH", Path.cwd()))
TARGET_BRANCH = os.getenv("TARGET_BRANCH", "main")
FORCE = os.getenv("FORCE", "0") == "1"
IGNORE_GLOBS = os.getenv("IGNORE_GLOBS", ".venv|.git|__pycache__|dist|build").split("|")
SEVERITY_THRESHOLD = os.getenv("SEVERITY_THRESHOLD", "error")

# Use project venv python if available
VENV_PY = REPO_PATH / ".venv" / "Scripts" / "python.exe"
if VENV_PY.exists():
    PYTHON = str(VENV_PY)
else:
    PYTHON = sys.executable

print({
    "python": sys.version,
    "platform": platform.platform(),
    "repo_path": str(REPO_PATH),
    "target_branch": TARGET_BRANCH,
    "force": FORCE,
    "python_exec": PYTHON,
})

# Ensure tools installed
REQS = [
    ("git", "git --version"),
    ("python", f"{PYTHON} --version"),
]
for name, cmd in REQS:
    try:
        out = subprocess.check_output(cmd, shell=True, text=True).strip()
        print(f"{name}: {out}")
    except Exception as e:
        print(f"WARN: {name} not found: {e}")

# Pip packages
PKGS = ["GitPython", "toml", "ruamel.yaml", "pytest", "coverage", "black", "isort", "ruff", "mypy", "bandit", "safety"]
for p in PKGS:
    try:
        __import__(p.replace("-", "_"))
    except Exception:
        print(f"Installing {p}...")
        subprocess.check_call([PYTHON, "-m", "pip", "install", p])

print("Toolchain ready.")


{'python': '3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:12) [MSC v.1935 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.26100-SP0', 'repo_path': 'c:\\Python Projects\\NoteBook_clean', 'target_branch': 'main', 'force': False, 'python_exec': 'c:\\Python Projects\\NoteBook_clean\\.venv\\Scripts\\python.exe'}
git: git version 2.50.1.windows.1
WARN: python not found: Command 'c:\Python Projects\NoteBook_clean\.venv\Scripts\python.exe --version' returned non-zero exit status 1.
Installing GitPython...
Toolchain ready.
Toolchain ready.


In [15]:
# Helper: enforce settings for write-enabled run and allow sync to proceed
DRY_RUN = False
FORCE = True
print("Overrides set:", {"DRY_RUN": DRY_RUN, "FORCE": FORCE})

Overrides set: {'DRY_RUN': False, 'FORCE': True}


In [16]:
# 1b. Target feature branch for refactor work
FEATURE_BRANCH = os.getenv("FEATURE_BRANCH", "refactor/two-pane")
TARGET_BRANCH = FEATURE_BRANCH
print({"TARGET_BRANCH": TARGET_BRANCH})

{'TARGET_BRANCH': 'refactor/two-pane'}


In [17]:
# 2. Sync From GitHub (Feature Branch)
from git import Repo, GitCommandError

repo = Repo(str(REPO_PATH))
assert not repo.bare, "Repo not found or bare"

# Local changes check
if repo.is_dirty(untracked_files=True) and not FORCE:
    raise SystemExit("Local changes present. Commit/stash or set FORCE=1 to proceed.")

origin = repo.remotes.origin if hasattr(repo.remotes, 'origin') else repo.create_remote('origin', url='origin')
try:
    # Ensure feature branch based on main
    base_branch = os.getenv("BASE_BRANCH", "main")
    repo.git.checkout(base_branch)
    origin.fetch()
    repo.git.pull('origin', base_branch)
    # Create/switch to feature branch
    if FEATURE_BRANCH in repo.branches:
        repo.git.checkout(FEATURE_BRANCH)
        repo.git.rebase(base_branch)
    else:
        repo.git.checkout('-b', FEATURE_BRANCH)
    print(f"On branch {FEATURE_BRANCH}, based on {base_branch}")
except GitCommandError as e:
    raise SystemExit(f"Git sync failed: {e}")


On branch refactor/two-pane, based on main


In [18]:
# 3. Discover Project Files to Modify
from pathlib import Path
import fnmatch

root = REPO_PATH
candidates = []
EXTS = {".py", ".ipynb", ".md", ".yaml", ".yml", ".toml", ".json"}

ignored = [p.strip() for p in IGNORE_GLOBS if p.strip()]

def _ignored(path: Path) -> bool:
    parts = set(path.parts)
    if any(part in {".git", "__pycache__"} for part in parts):
        return True
    s = str(path)
    for pat in ignored:
        if pat and pat in s:
            return True
    return False

for p in root.rglob('*'):
    if p.is_file() and p.suffix.lower() in EXTS and not _ignored(p):
        candidates.append(p)

print(f"Found {len(candidates)} candidate files.")
print("Sample:")
for p in candidates[:20]:
    print(" -", p.relative_to(root))


Found 283 candidate files.
Sample:
 - db_access.py
 - db_pages.py
 - db_sections.py
 - db_version.py
 - import.py
 - inspect_db.py
 - inspect_sections.py
 - main.py
 - media_store.py
 - pyproject.toml
 - README.md
 - RefactorPlan.ipynb
 - sample_spreadsheet.py
 - settings.json
 - settings_manager.py
 - ui_loader.py
 - ui_logic.py
 - ui_richtext.py
 - ui_sections.py
 - ui_tabs.py


In [ ]:
# 4. Define Safe Transformations (Skip Any 'Binder' Renames)
import re
from difflib import unified_diff

# Prefer in-notebook DRY_RUN flag; fallback to env
try:
    dry_run = bool(DRY_RUN)
except NameError:
    import os as _os
    dry_run = _os.getenv("DRY_RUN", "1") == "1"

# Example transform rules (add more as needed)
def to_fstrings(text: str) -> str:
    # Minimal example: replace str.format with f-strings where trivial
    return text  # placeholder no-op

def standardize_logging(text: str) -> str:
    return text  # stub: add real rules later

TRANSFORMS = [to_fstrings, standardize_logging]

BINDER_GUARD = re.compile(r"\b[Bb]inder\b")

def apply_transforms(path: Path, text: str) -> str:
    original = text
    for fn in TRANSFORMS:
        text = fn(text)
    # Guard: if diff would alter any line containing Binder tokens, skip
    if BINDER_GUARD.search(original) or BINDER_GUARD.search(text):
        if original != text:
            diff = list(unified_diff(original.splitlines(True), text.splitlines(True)))
            if any(BINDER_GUARD.search(line) for line in diff):
                return original
    return text

print("Transforms ready. Dry run:", dry_run)


Transforms ready. Dry run: True


In [7]:
# 5. Apply Transformations With Backups and Diffs
from difflib import unified_diff
import json

report = {"changed": [], "skipped": [], "errors": []}

INCLUDE = [g.strip() for g in os.getenv("INCLUDE_GLOBS", "").split(",") if g.strip()]
EXCLUDE = [g.strip() for g in os.getenv("EXCLUDE_GLOBS", "").split(",") if g.strip()]

def matched(path: Path, globs):
    s = str(path)
    return any(fnmatch.fnmatch(s, g) for g in globs)

for path in candidates:
    try:
        if INCLUDE and not matched(path, INCLUDE):
            continue
        if EXCLUDE and matched(path, EXCLUDE):
            continue
        if path.suffix.lower() not in {".py", ".md", ".yaml", ".yml", ".toml", ".json"}:
            continue
        text = path.read_text(encoding="utf-8", errors="ignore")
        new_text = apply_transforms(path, text)
        if new_text != text:
            print(f"\n--- {path} \n")
            diff = "".join(unified_diff(text.splitlines(True), new_text.splitlines(True), fromfile=str(path), tofile=str(path)))
            print(diff)
            if not dry_run:
                bak = path.with_suffix(path.suffix + ".bak")
                bak.write_text(text, encoding="utf-8")
                path.write_text(new_text, encoding="utf-8")
                report["changed"].append(str(path))
        else:
            report["skipped"].append(str(path))
    except Exception as e:
        report["errors"].append({"file": str(path), "error": str(e)})

print("\nReport:")
print(json.dumps(report, indent=2))



Report:
{
  "changed": [],
  "skipped": [
    "c:\\Python Projects\\NoteBook_clean\\db_access.py",
    "c:\\Python Projects\\NoteBook_clean\\db_pages.py",
    "c:\\Python Projects\\NoteBook_clean\\db_sections.py",
    "c:\\Python Projects\\NoteBook_clean\\db_version.py",
    "c:\\Python Projects\\NoteBook_clean\\import.py",
    "c:\\Python Projects\\NoteBook_clean\\inspect_db.py",
    "c:\\Python Projects\\NoteBook_clean\\inspect_sections.py",
    "c:\\Python Projects\\NoteBook_clean\\main.py",
    "c:\\Python Projects\\NoteBook_clean\\media_store.py",
    "c:\\Python Projects\\NoteBook_clean\\README.md",
    "c:\\Python Projects\\NoteBook_clean\\sample_spreadsheet.py",
    "c:\\Python Projects\\NoteBook_clean\\settings.json",
    "c:\\Python Projects\\NoteBook_clean\\settings_manager.py",
    "c:\\Python Projects\\NoteBook_clean\\ui_loader.py",
    "c:\\Python Projects\\NoteBook_clean\\ui_logic.py",
    "c:\\Python Projects\\NoteBook_clean\\ui_richtext.py",
    "c:\\Python Projects\\

In [8]:
# 6. Update Tooling Config (pyproject.toml, linters, CI)
import toml
from ruamel.yaml import YAML

def _safe_load_toml(path: Path):
    data = {}
    if path.exists():
        try:
            data = toml.loads(path.read_text(encoding="utf-8"))
        except Exception:
            pass
    return data

def _safe_dump_toml(path: Path, data: dict):
    tmp = path.with_suffix(path.suffix + ".tmp")
    tmp.write_text(toml.dumps(data), encoding="utf-8")
    tmp.replace(path)

pyproj = REPO_PATH / "pyproject.toml"
proj = _safe_load_toml(pyproj)
proj.setdefault("tool", {}).setdefault("black", {"line-length": 100})
proj["tool"].setdefault("isort", {"profile": "black"})
proj["tool"].setdefault("ruff", {"line-length": 100})
proj["tool"].setdefault("mypy", {"ignore_missing_imports": True})
_safe_dump_toml(pyproj, proj)

# Minimal CI YAML touch (idempotent)
yaml = YAML()
ci_path = REPO_PATH / ".github" / "workflows" / "ci.yml"
ci = {}
if ci_path.exists():
    try:
        ci = yaml.load(ci_path.read_text(encoding="utf-8")) or {}
    except Exception:
        ci = {}
ci.setdefault("name", "CI")
ci.setdefault("on", ["push", "pull_request"])  # minimal
jobs = ci.setdefault("jobs", {})
jobs.setdefault("lint", {"runs-on": "ubuntu-latest", "steps": [{"uses": "actions/checkout@v4"}, {"run": "pip install black isort ruff && ruff . && black --check . && isort --check-only ."}]})
jobs.setdefault("test", {"runs-on": "ubuntu-latest", "steps": [{"uses": "actions/checkout@v4"}, {"run": "pip install -r requirements.txt pytest && pytest -q"}]})
ci_path.parent.mkdir(parents=True, exist_ok=True)
yaml.dump(ci, ci_path.open("w", encoding="utf-8"))
print("Tooling config updated.")


Tooling config updated.


In [ ]:
# 7. Format (check-only) and Lint
import subprocess, sys, shlex

# Use project venv python when available
try:
    PYTHON
except NameError:
    from pathlib import Path as _Path
    _venv = _Path.cwd() / ".venv" / "Scripts" / "python.exe"
    PYTHON = str(_venv) if _venv.exists() else sys.executable

def run_cmd(cmd: str) -> int:
    print("Running:", cmd)
    try:
        result = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
        if result.stdout:
            print(result.stdout)
        if result.stderr:
            print(result.stderr, file=sys.stderr)
        return result.returncode
    except Exception as e:
        print(f"Error executing {cmd}: {e}", file=sys.stderr)
        return 1

BLACK = f"{PYTHON} -m black {'--check' if dry_run else ''} .".strip()
ISORT = f"{PYTHON} -m isort {'--check-only' if dry_run else ''} .".strip()
RUFF = f"{PYTHON} -m ruff {'check --exit-zero' if dry_run else 'check'} .".strip()

rc_black = run_cmd(BLACK)
rc_isort = run_cmd(ISORT)
rc_ruff = run_cmd(RUFF)

print("Lint summary:", {"black": rc_black, "isort": rc_isort, "ruff": rc_ruff})

Running: c:\Users\donal\AppData\Local\Programs\Python\Python311\python.exe -m black --check .
Running: c:\Users\donal\AppData\Local\Programs\Python\Python311\python.exe -m isort --check-only .
Running: c:\Users\donal\AppData\Local\Programs\Python\Python311\python.exe -m ruff check --exit-zero .
Lint summary: {'black': 1, 'isort': 1, 'ruff': 1}


Error executing c:\Users\donal\AppData\Local\Programs\Python\Python311\python.exe -m black --check .: [WinError 2] The system cannot find the file specified
Error executing c:\Users\donal\AppData\Local\Programs\Python\Python311\python.exe -m isort --check-only .: [WinError 2] The system cannot find the file specified
Error executing c:\Users\donal\AppData\Local\Programs\Python\Python311\python.exe -m ruff check --exit-zero .: [WinError 2] The system cannot find the file specified


In [ ]:
# 8. Run Unit Tests and Coverage
import sys, subprocess

try:
    PYTHON
except NameError:
    from pathlib import Path as _Path
    _venv = _Path.cwd() / ".venv" / "Scripts" / "python.exe"
    PYTHON = str(_venv) if _venv.exists() else sys.executable

cmd = f"{PYTHON} -m pytest -q --maxfail=1"
rc = subprocess.call(cmd, shell=True)
print("pytest rc:", rc)

# Coverage summary if available
cov_xml = REPO_PATH / "coverage.xml"
if cov_xml.exists():
    import xml.etree.ElementTree as ET
    root = ET.parse(str(cov_xml)).getroot()
    summaries = root.findall('.//coverage')
    print("coverage summaries:", summaries)


pytest rc: 5


In [ ]:
# 9. Type and Security checks
import subprocess, sys, shlex

try:
    PYTHON
except NameError:
    from pathlib import Path as _Path
    _venv = _Path.cwd() / ".venv" / "Scripts" / "python.exe"
    PYTHON = str(_venv) if _venv.exists() else sys.executable

def run_cmd(cmd: str) -> int:
    print("Running:", cmd)
    try:
        result = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
        if result.stdout:
            print(result.stdout)
        if result.stderr:
            print(result.stderr, file=sys.stderr)
        return result.returncode
    except Exception as e:
        print(f"Error executing {cmd}: {e}", file=sys.stderr)
        return 1

MYPY = f"{PYTHON} -m mypy ."
BANDIT = f"{PYTHON} -m bandit -q -r ."
SAFETY = f"{PYTHON} -m safety check --full-report"

rc_mypy = run_cmd(MYPY)
rc_bandit = run_cmd(BANDIT)
rc_safety = run_cmd(SAFETY)

print("Checks summary:", {"mypy": rc_mypy, "bandit": rc_bandit, "safety": rc_safety})

Running: c:\Users\donal\AppData\Local\Programs\Python\Python311\python.exe -m mypy .
Running: c:\Users\donal\AppData\Local\Programs\Python\Python311\python.exe -m bandit -q -r .
Running: c:\Users\donal\AppData\Local\Programs\Python\Python311\python.exe -m safety check --full-report
Checks summary: {'mypy': 1, 'bandit': 1, 'safety': 1}


Error executing c:\Users\donal\AppData\Local\Programs\Python\Python311\python.exe -m mypy .: [WinError 2] The system cannot find the file specified
Error executing c:\Users\donal\AppData\Local\Programs\Python\Python311\python.exe -m bandit -q -r .: [WinError 2] The system cannot find the file specified
Error executing c:\Users\donal\AppData\Local\Programs\Python\Python311\python.exe -m safety check --full-report: [WinError 2] The system cannot find the file specified


In [ ]:
# 10. Bump Version and Update Changelog
import datetime

def bump_version_in_pyproject(pyproj: Path):
    data = _safe_load_toml(pyproj)
    tool_poetry = data.setdefault("tool", {}).setdefault("poetry", {})
    ver = tool_poetry.get("version") or data.setdefault("project", {}).get("version")
    if not ver:
        return None
    parts = ver.split('.')
    parts[-1] = str(int(parts[-1]) + 1)
    new_ver = '.'.join(parts)
    if "version" in tool_poetry:
        tool_poetry["version"] = new_ver
    else:
        data.setdefault("project", {})["version"] = new_ver
    _safe_dump_toml(pyproj, data)
    return new_ver

new_ver = bump_version_in_pyproject(pyproj)
print("New version:", new_ver)

# Basic changelog from git log
log = subprocess.check_output(["git", "log", "--pretty=format:%h %s", "-n", "20"], cwd=str(REPO_PATH), text=True)
changelog_path = REPO_PATH / "CHANGELOG.md"
ts = datetime.datetime.utcnow().strftime("%Y-%m-%d")
entry = f"\n\n## {new_ver or 'Unreleased'} - {ts}\n" + '\n'.join(f"- {line}" for line in log.splitlines())
with changelog_path.open("a", encoding="utf-8") as f:
    f.write(entry)
print("Changelog updated.")


In [ ]:
# 10b. Tag current main as stable (optional, idempotent)
try:
    from git import Repo
    repo = Repo(str(REPO_PATH))
    base_branch = os.getenv("BASE_BRANCH", "main")
    tag_name = os.getenv("STABLE_TAG", f"stable-{__import__('datetime').datetime.utcnow().strftime('%Y%m%d')}")
    # Only create tag if it doesn't exist
    if tag_name not in [t.name for t in repo.tags]:
        cur = repo.git.rev_parse(f"origin/{base_branch}")
        repo.create_tag(tag_name, cur, message=f"Stable snapshot of {base_branch}")
        print(f"Created tag {tag_name} at {cur}")
    else:
        print(f"Tag {tag_name} already exists; skipping")
except Exception as e:
    print("WARN: Could not create stable tag:", e)


In [ ]:
# 11. Commit, Tag, and Push to GitHub (Feature Branch)
from git import Repo

repo = Repo(str(REPO_PATH))
repo.git.add(A=True)
msg = "chore(refactor): two-pane groundwork (guarded transforms, tooling updates, Binder retained)"
if 'new_ver' in globals() and new_ver:
    msg += f"\n\nBump version to {new_ver}"
repo.index.commit(msg)
if 'new_ver' in globals() and new_ver:
    # tag release on feature branch too (optional)
    tag_name = f"v{new_ver}"
    if tag_name not in [t.name for t in repo.tags]:
        repo.create_tag(tag_name, message=f"Release {new_ver}")
repo.remotes.origin.push()
# Push feature branch explicitly
repo.remotes.origin.push(refspec=f"HEAD:{FEATURE_BRANCH}")
# Push tags if any
if 'new_ver' in globals() and new_ver:
    repo.remotes.origin.push(tags=True)
print("Pushed changes. HEAD:", repo.head.commit.hexsha, "branch:", FEATURE_BRANCH)
